# **Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential  # type: ignore
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences   # type: ignore
from tensorflow.keras.regularizers import l2    # type: ignore
from tensorflow.keras.optimizers import Adam    # type: ignore
import shap
import lime
from sklearn.impute import SimpleImputer

# **File path and dataframe**

In [2]:
df = pd.read_excel(r"F:\University\Uni Stuff (semester 11)\Thesis\code\data\raw\thesis_excel (4).xlsx", parse_dates=['Crime date'])

In [4]:
df_agg = df.set_index('Crime date').groupby(['Crime Location', pd.Grouper(freq='D')])['Allegation'].count().reset_index()
df_agg = df_agg.rename(columns={'Allegation': 'Crime count'})

In [7]:
df_pivot = df_agg.pivot(index='Crime date', columns='Crime Location', values='Crime count').fillna(0)
# Ensure the date index is complete (no missing days)
df_pivot = df_pivot.resample('D').asfreq(fill_value=0)
print(df_pivot.head())

Crime Location  daily time building  kala paharia  kasipur  matuail  nahati  \
Crime date                                                                    
2024-01-02                     13.0          33.0      0.0      0.0     0.0   
2024-01-03                      0.0           0.0      0.0      0.0     0.0   
2024-01-04                      0.0           0.0      0.0      0.0     0.0   
2024-01-05                      0.0           0.0      0.0      0.0     0.0   
2024-01-06                      0.0           0.0      0.0      0.0     0.0   

Crime Location  narayangonj sadar  sadipur  sector#7  sonargaon  vatipara  \
Crime date                                                                  
2024-01-02                    0.0      0.0       3.0        0.0       0.0   
2024-01-03                    0.0      0.0       0.0        0.0       0.0   
2024-01-04                    0.0      0.0       0.0        0.0       0.0   
2024-01-05                    0.0      0.0       0.0        0